In [ ]:
import numpy as np
from ctypes import *
import socket
import errno

In [ ]:
NBUF = 16
BLK_SIZE = 1024*16*1000
DATA_SIZE = BLK_SIZE * NBUF
HOST = 'localhost'
PORT = 5001
NBLK = 16

intensity = np.memmap('/dev/hugepages/fpga0.bin', dtype='float32', shape=(DATA_SIZE))
intensity.shape

blk_ptr = cast(c_void_p(intensity.ctypes.data + DATA_SIZE * 4), POINTER(c_long))

In [ ]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen(1)

    n = 0
    while True:
        conn, addr = s.accept()
        print('Connected by', addr)

        while True:
            data = intensity[BLK_SIZE*n:BLK_SIZE*(n+1)]
            try:
                conn.sendall(data)
            except socket.error as e:
                if e.errno != errno.ECONNRESET:
                    raise
                print('Connection closed for', addr)
                break

            print(f'Block #{n} sent')
            n = (n + 1) % NBUF

In [ ]:
blk_ptr.contents.value

In [ ]:
last_blk = blk_ptr.contents.value

while (True):
    new_blk = blk_ptr.contents.value
    if (new_blk != last_blk):
        intensity[blk_size*last_blk:blk_size*(last_blk+1)]

        last_blk += 1
        if (last_blk >= nbuf):
            last_blk = 0




In [ ]:
libc = CDLL("libc.so.6")

In [ ]:
libc.time()

In [ ]:
while (True):
    pass